# Pyro models

In [ ]:
import pandas as pd
import numpy as np
from hmmlearn import hmm
import matplotlib.pyplot as plt
from pyprojroot import here
import pyro
import pyro.distributions as dist
import pyro.distributions.constraints as constraints

data = pd.read_csv(here("data/recent_donations.csv"))
data

In [ ]:
def simple_pyro_model():
    FY = pyro.sample(name="First donation year",
                     fn=dist.Uniform(low=2010, high=2020).expand_by([10]).to_event(1)),
                     obs=data["First donation year"].values)